# Online Adapt

init enrollment만 adaptation에 쓰인다고 가정해야, 효율적으로 구현할 수 있다.

In [197]:
%load_ext autoreload
%autoreload 2
%pylab
%matplotlib inline

import os
import sys
import pandas as pd
import pickle
import numpy as np
from tqdm import tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [202]:
from batch_sv_system import get_embeds, cosine_sim, score_trials
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_fscore_support

In [199]:
from utils import key2df, df2dict, compute_eer, get_id2idx

embed_dir = "embeddings/voxc2_fbank64_voxc2untied_xvector/"
sv_embeds = np.load(embed_dir+"ln_lda_sv_embeds.npy")
sv_keys = pickle.load(open(embed_dir + "/sv_keys.pkl", "rb"))
sv_id2idx = get_id2idx(sv_keys)

cohort_ids = np.load("trials/dev940_eval311/dev_cohort_ids.npy")
cohort_embeds = get_embeds(cohort_ids, sv_embeds, sv_id2idx, norm=False)

In [205]:
from ioffe_plda.verifier import Verifier
py_plda_model = Verifier()
py_plda_model = Verifier(pickle.load(open("py_plda_model_ln_lda.pkl", "rb")))

In [208]:
def score_trials(enr_embeds, test_embeds, score_method):
        if score_method == "scoreAvg":
            score = py_plda_model.score_avg(enr_embeds, test_embeds).mean(0)
        elif score_method == "vectorAvg":
            score = py_plda_model.vector_avg(enr_embeds, test_embeds).mean(0)
        elif score_method == "multiSessScale":
            score = py_plda_model.multi_sess(enr_embeds, test_embeds, cov_scaling=True).mean(0)
        elif score_method == "multiSessAdapt":
            score = py_plda_model.multi_sess(enr_embeds, test_embeds, cov_adapt=True).mean(0)
        else:
            raise NotImplemtedError
        
        return score

## Online Adaptation

In [223]:
trials = pickle.load(open("trials/dev940_eval311/hard_enr3xsess_ntar9/trials.pkl", "rb"))

In [ ]:
adapt_ratio = 0.2

eT_list =  [-4.26017]
init = []
optimal = []
labels = []
scoreAvg = {k:[] for k in eT_list}
vectorAvg = {k:[] for k in eT_list}
multiSessScale = {k:[] for k in eT_list}
multiSessAdapt = {k:[] for k in eT_list}

for t_i in tqdm(range(len(trials)), total=len(trials)):
    enr_spk, enr_ids, test_trial = trials[t_i]
    test_trial = (np.array(test_trial.id), np.array(test_trial.label))

    adapt_len = int(len(test_trial[0]) * adapt_ratio)
    
    init_enr_embeds = get_embeds(enr_ids, sv_embeds,  sv_id2idx, norm=False)
    test_embeds = get_embeds(test_trial[0], sv_embeds, sv_id2idx, norm=False)

    adapt_scores = score_trials(init_enr_embeds, test_embeds, "scoreAvg")
    
    for eT in eT_list:
        adapt_times = np.nonzero(adapt_scores > eT)[0]
        adapted_embeds = test_embeds[adapt_scores > eT]
        
        scoreAvg_list = []
        vectorAvg_list = []
        multiSessScale_list = []
        multiSessAdapt_list = []
        
        prev_t = 0
        for n, t in enumerate(adapt_times.tolist() + [len(test_embeds)]):
#             print(n, prev_t, t)
            total_enr_embeds = np.concatenate([init_enr_embeds, adapted_embeds[:n]])
            test_embeds_ = test_embeds[prev_t:t]
#             scoreAvg_list.append(score_trials(total_enr_embeds, test_embeds_, "scoreAvg_list"))
            vectorAvg_list.append(score_trials(total_enr_embeds, test_embeds_, "vectorAvg"))
            multiSessScale_list.append(score_trials(total_enr_embeds, test_embeds_, "multiSessScale"))
            multiSessAdapt_list.append(score_trials(total_enr_embeds, test_embeds_, "multiSessAdapt"))
            prev_t = t
    
        init.append(score_trials(init_enr_embeds, test_embeds, "multiSessScale")[adapt_len:])
#         scoreAvg[eT].append(np.concatenate(scoreAvg_list)[adapt_len:])
        vectorAvg[eT].append(np.concatenate(vectorAvg)[adapt_len:])
        multiSessScale[eT].append(np.concatenate(multiSessScale)[adapt_len:])
        multiSessAdapt[eT].append(np.concatenate(multiSessAdapt)[adapt_len:])
        
    labels.append(test_trial[1])
    break
        

In [ ]:
for eT in eT_list:
#     print("optimal")
#     compute_eer(np.concatenate(online_optimal), np.concatenate(online_labels))
#     print("init")
#     compute_eer(np.concatenate(online_init), np.concatenate(online_labels))
#     print("scoreAvg")
#     compute_eer(np.concatenate(online_scoreAvg[eT]), np.concatenate(online_labels))
    print("vectorAvg")
    compute_eer(np.concatenate(online_vectorAvg[eT]), np.concatenate(online_labels));
    print("multiSessScale")
    compute_eer(np.concatenate(online_multiSessScale[eT]), np.concatenate(online_labels))
    print("multiSessAdapt")
    compute_eer(np.concatenate(online_multiSessAdapt[eT]), np.concatenate(online_labels))
    print("multiSessScaleAdapt")
    compute_eer(np.concatenate(online_multiSessScaleAdapt[eT]), np.concatenate(online_labels))